In [1]:
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, r2_score, root_mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import optuna

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os 
import warnings
warnings.filterwarnings("ignore")

from utils import *
from heuristics import *

DATA_PATH = f"{os.getcwd()}/data"

df_heating = pd.read_excel(f"{DATA_PATH}/Heating_degree_days_by_census_division.xls", skiprows=10)
df_cooling = pd.read_excel(f"{DATA_PATH}/Cooling_degree_days_by_census_division.xls", skiprows=10)
df_consumption = pd.read_excel(f"{DATA_PATH}/Residential_sector_energy_consumption.xls", skiprows=10)
df_average_prices = pd.read_excel(f"{DATA_PATH}/Average_prices_of_electricity_to_ultimate_customers.xls", skiprows=10)

df_consumption = df_consumption[["Month", "Total Energy Consumed by the Residential Sector"]]
df_average_prices = df_average_prices[["Month", "Average Price of Electricity to Ultimate Customers, Total", "Average Price of Electricity to Ultimate Customers, Residential"]]

merged_df = pd.merge(df_heating, df_cooling, on='Month', how='inner')
merged_df = pd.merge(merged_df, df_consumption, on='Month', how='inner')
merged_df = pd.merge(merged_df, df_average_prices, on='Month', how='inner')
merged_df = merged_df.drop(0)
df_cleaned = merged_df[~merged_df.isin(['Not Available']).any(axis=1)]
df_cleaned = df_cleaned.drop(columns=["Month"])

X, y = df_cleaned.drop(columns=["Total Energy Consumed by the Residential Sector"]), df_cleaned["Total Energy Consumed by the Residential Sector"]

#scaler = StandardScaler()
#X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

/home/ax-pedro-camara/miniconda3/envs/tcc/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ga = GeneticAlgorithm(
#     pop_size=20,
#     crossover_rate=0.8,
#     mutation_rate=0.1,
#     n_generations=10,
# )

# params, best = ga.run(
#     X,
#     y,
# )

In [3]:
from pymoo.algorithms.soo.nonconvex.de import DE
from pymoo.optimize import minimize

problem = HyperparameterOptimizationProblem(X, y)
de = DE(
    pop_size=20,
    variant="DE/best/1/bin",
    CR=0.8,
    F=0.7,
    jitter=False,
    dither="vector",
)
res = minimize(
    problem,
    de,
    ("n_gen", 10)
)

In [9]:
res.F

array([-0.72105477])

In [4]:
# de = DifferentialEvolution(
#     pop_size=30,
#     crossover_rate=0.8,
#     mutation_rate=0.7,
#     n_generations=10,
# )

# params, best = de.run(
#     X,
#     y,
# )

In [5]:
# optuna_mlp = OptunaMLP(X, y, n_trials=20)
# best_trial = optuna_mlp.run()

In [6]:
best_trial

NameError: name 'best_trial' is not defined

In [ ]:
params

In [ ]:
mlp = best #MLPRegressor(hidden_layer_sizes=(20, 15, 10), max_iter=200)
metrics = ["r2", "neg_root_mean_squared_error", "neg_mean_absolute_error"]

In [ ]:
scores_mlp = train_with_cross_val(mlp, X, y, metrics)
mlp.fit(X_train, y_train)

In [ ]:
r2_score(y_test, mlp.predict(X_test))

In [ ]:
root_mean_squared_error(y_test, mlp.predict(X_test))

In [ ]:
for metric in ["r2", "neg_root_mean_squared_error", "neg_mean_absolute_error"]:
    print(f"MLP {metric}: {scores_mlp[metric].mean()}")

In [ ]:
scores_mlp

In [ ]:
for metric, scores in scores_mlp.items():
    print(f"{metric}: {scores.mean():.4f} ± {scores.std():.4f}")